First, import needed modules

In [109]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from shapely.geometry import LineString, Polygon
import os

Initalize variables

In [110]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Load Dataset which has the endpoints of power line projects

In [111]:
powerlines = pd.read_csv(f'{path}/data/geodata/powerlines.csv', sep=';')
#powerlines = powerlines.set_index("# BBPlG 2013", drop = False)
powerlines.head()

,# BBPlG 2013,Start,End,New,Retro
0,3,Brunsbüttel,Großgartach,1,NaN
1,4,Wilster,Grafenrheinfeld,1,NaN
2,5,Bad Lauchstädt,Meitingen,1,NaN
3,17,Mecklar,Grafenrheinfeld,1,NaN


Get the geocoordinates of the 'endpoint' cities using OSM/Nominatim API

In [112]:
start_locations = geocode(powerlines['Start'], provider='nominatim', user_agent='plm', timeout=4)
end_locations = geocode(powerlines['End'], provider='nominatim', user_agent='plm', timeout=4)

Merge into powerlines dataset

In [113]:
powerlines_geo_start = start_locations.join(powerlines, how = 'left')
powerlines_geo_start.rename(columns = {'geometry':'Start_geometry', 'address':'Start_address'}, inplace = True)
powerlines_geo = end_locations.join(powerlines_geo_start, how = 'left')
powerlines_geo.rename(columns = {'geometry':'End_geometry', 'address':'End_address'}, inplace = True)
powerlines_geo = powerlines_geo.reindex(columns=['# BBPlG 2013', 'Start', 'Start_geometry', 'Start_address', 'End', 'End_geometry', 'End_address', 'direct_line', 'New', 'Retro'])
powerlines_geo.head()

,# BBPlG 2013,Start,Start_geometry,Start_address,End,End_geometry,End_address,direct_line,New,Retro
0,3,Brunsbüttel,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",Großgartach,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ...",NaN,1,NaN
1,4,Wilster,POINT (9.37338 53.92328),"Wilster, Kreis Steinburg, Schleswig-Holstein, ...",Grafenrheinfeld,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...",NaN,1,NaN
2,5,Bad Lauchstädt,POINT (11.86870 51.38813),"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl...",Meitingen,POINT (10.85000 48.55000),"Meitingen, Landkreis Augsburg, Bayern, 86405, ...",NaN,1,NaN
3,17,Mecklar,POINT (9.76125 50.91746),"Mecklar, Ludwigsau, Landkreis Hersfeld-Rotenbu...",Grafenrheinfeld,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...",NaN,1,NaN


Create direct line from start and end coordinates

In [114]:
for i in range (0, len(powerlines_geo)):
    start = powerlines_geo['Start_geometry'].loc[powerlines_geo.index[i]]
    end = powerlines_geo['End_geometry'].loc[powerlines_geo.index[i]]
    print(start, end)
    line = LineString([start, end])
    powerlines_geo['direct_line'].loc[powerlines_geo.index[i]] = line
powerlines_geo.head()

POINT (9.1395423 53.8972549) POINT (9.1258701 49.1425406)
POINT (9.373378722135374 53.9232825) POINT (10.1966808 50.0055317)
POINT (11.8686979 51.3881334) POINT (10.85 48.55)
POINT (9.7612468 50.9174645) POINT (10.1966808 50.0055317)


/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_25057/2805498544.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  powerlines_geo['direct_line'].loc[powerlines_geo.index[i]] = line
/opt/miniconda3/envs/python310/lib/python3.10/site-packages/pandas/core/internals/blocks.py:930: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return self.coerce_to_target_dtype(value).setitem(indexer, value)
/opt/miniconda3/envs/python310/lib/python3.10/site-packages/pandas/core/internals/managers.py:304: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  applied = getattr(b, f)(**kwargs)


,# BBPlG 2013,Start,Start_geometry,Start_address,End,End_geometry,End_address,direct_line,New,Retro
0,3,Brunsbüttel,POINT (9.13954 53.89725),"Brunsbüttel, Dithmarschen, Schleswig-Holstein,...",Großgartach,POINT (9.12587 49.14254),"Großgartach, Leingarten, Landkreis Heilbronn, ...","LINESTRING (9.1395423 53.8972549, 9.1258701 49...",1,NaN
1,4,Wilster,POINT (9.37338 53.92328),"Wilster, Kreis Steinburg, Schleswig-Holstein, ...",Grafenrheinfeld,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...","LINESTRING (9.373378722135374 53.9232825, 10.1...",1,NaN
2,5,Bad Lauchstädt,POINT (11.86870 51.38813),"Bad Lauchstädt, Lindenstraße, Lindenstraße, Kl...",Meitingen,POINT (10.85000 48.55000),"Meitingen, Landkreis Augsburg, Bayern, 86405, ...","LINESTRING (11.8686979 51.3881334, 10.85 48.55)",1,NaN
3,17,Mecklar,POINT (9.76125 50.91746),"Mecklar, Ludwigsau, Landkreis Hersfeld-Rotenbu...",Grafenrheinfeld,POINT (10.19668 50.00553),"Grafenrheinfeld, Landkreis Schweinfurt, Bayern...","LINESTRING (9.7612468 50.9174645, 10.1966808 5...",1,NaN
